In [ ]:
import glob

listOfFileNames = []
listOfClassNames = []
listOfDataPerClass = []
files = []
classNames = []

classNames = glob.glob("/Users/user/Desktop/Thesis/placeholder/*")

for c in classNames:
    className = c.split("\\")[1]
    listOfClassNames.append(c.split("\\")[1])
    files = glob.glob("/Users/user/Desktop/Thesis/placeholder/"+className+"/*.csv")
    listOfFileNames = []
    for f in files:
        listOfFileNames.append(f.split("\\")[1])
    listOfDataPerClass.append(listOfFileNames)

    
#listOfClassNames
#listOfDataPerClass[0]
print("Done listing the class and file names")

In [ ]:
#len(listOfDataPerClass)
#listOfDataPerClass[3]

import pandas as pd


listOfDataFramesPerClass = {}
dataFrameDictionary = {}
dataFrames = []

for c, d in zip(listOfClassNames, listOfDataPerClass):
    print("----------------- Word " + c + "-----------------")
    emgDataFrames = []
    accDataFrames = []
    gyroDataFrames = []

    dataFrameDictionary = {}
    className = c
    ctr = 0
    for fileName in d:
        fileLocation = "/Users/user/Desktop/Thesis/placeholder/"+className+"/"+fileName + ""
        df = pd.read_csv(fileLocation)
        
        if "emg" in fileName:            
            emgDataFrames.append(df)
            ctr+=1
        elif ("accelerometer") in fileName:
            accDataFrames.append(df)
            ctr+=1
        elif ("gyro") in fileName:
            gyroDataFrames.append(df)
            ctr+=1
    
    dataFrameDictionary["emg"] = emgDataFrames
    dataFrameDictionary["acc"] = accDataFrames
    dataFrameDictionary["gyro"] = gyroDataFrames

    listOfDataFramesPerClass[className] = dataFrameDictionary
    print("Total of " + str(ctr) + " files")


print("Done converting to dataframes!")


In [ ]:
len(imuDataFrames)

## For EMG Values

# Trimming

In [ ]:
import pandas as pd

listOfTrimmedDataFramesPerClass = {}

for k in listOfDataFramesPerClass.keys():
    print("----------------- Word " + k + "-----------------")
    ctr = 0
    trimmedDataFramesPerClass = []
    dataFrameDictionary = {}

    for d in listOfDataFramesPerClass[k]['emg']:
        keep_col = ['emg1','emg2','emg3','emg4','emg5','emg6','emg7','emg8']
        new_df = d[keep_col]
        new_df = new_df[:-49]
        trimmed_df = new_df.iloc[49:]
        if len(trimmed_df) < 100:
            print(str(ctr) + " " + k)
        trimmedDataFramesPerClass.append(trimmed_df)
        ctr += 1
        
    print("Done Trimming " + str(ctr) + " files")
    dataFrameDictionary["emg"] = trimmedDataFramesPerClass
    listOfTrimmedDataFramesPerClass[k] = dataFrameDictionary

# Normalization

In [ ]:
from sklearn import preprocessing

listOfNormalizedDataFramesPerClass = {}

for k in listOfTrimmedDataFramesPerClass.keys():
    print("----------------- Word " + k + "-----------------")
    ctr = 0
    NormalizedDataFramesPerClass = []
    dataFrameDictionary = {}

    for d in listOfTrimmedDataFramesPerClass[k]['emg']:
        #print(str(len(d.index)) + " " + str(ctr))
        # Create x, where x the 'scores' column's values as floats
        x = d.values.astype(float)

        # Create a minimum and maximum processor object
        min_max_scaler = preprocessing.MinMaxScaler()

        # Create an object to transform the data to fit minmax processor
        x_scaled = min_max_scaler.fit_transform(x)

        # Run the normalizer on the dataframe
        df_normalized = pd.DataFrame(x_scaled)
        #df_normalized
        NormalizedDataFramesPerClass.append(df_normalized)
        ctr += 1
        
    print("Done Normalizing " + str(ctr) + " files")
    dataFrameDictionary["emg"] = NormalizedDataFramesPerClass
    listOfNormalizedDataFramesPerClass[k] = dataFrameDictionary  
    
print("Data Normalization done!!")



Compute Features

In [ ]:
import pandas as pd
from statsmodels import robust
import numpy as np

listOfVarianceDataFramePerClass = []
listOfMavDataFramePerClass = []
listOfMeanDataFramePerClass = []
listOfSdDataFramePerClass = []
listOfRmsDataFramePerClass = []

for k in listOfNormalizedDataFramesPerClass.keys():
    print("----------------- Word " + k + "-----------------")
    ctr = 0

    varianceDataFramePerClass = []
    varianceDataFrame = pd.DataFrame(columns = ['Variance of EMG1', 'Variance of EMG2', 'Variance of EMG3', 'Variance of EMG4',
                                 'Variance of EMG5', 'Variance of EMG6', 'Variance of EMG7', 'Variance of EMG8'])
    mavDataFramePerClass = []
    mavDataFrame = pd.DataFrame(columns = ['Mav of EMG1', 'Mav of EMG2', 'Mav of EMG3', 'Mav of EMG4',
                                 'Mav of EMG5', 'Mav of EMG6', 'Mav of EMG7', 'Mav of EMG8'])
    meanDataFramePerClass = []
    meanDataFrame = pd.DataFrame(columns = ['Mean of EMG1', 'Mean of EMG2', 'Mean of EMG3', 'Mean of EMG4',
                                 'Mean of EMG5', 'Mean of EMG6', 'Mean of EMG7', 'Mean of EMG8'])
    sdDataFramePerClass = []
    sdDataFrame = pd.DataFrame(columns = ['Sd of EMG1', 'Sd of EMG2', 'Sd of EMG3', 'Sd of EMG4',
                                 'Sd of EMG5', 'Sd of EMG6', 'Sd of EMG7', 'Sd of EMG8'])
    rmsDataFramePerClass = []
    rmsDataFrame = pd.DataFrame(columns = ['Rms of EMG1', 'Rms of EMG2', 'Rms of EMG3', 'Rms of EMG4',
                                 'Rms of EMG5', 'Rms of EMG6', 'Rms of EMG7', 'Rms of EMG8'])
    
    for df_normalized in listOfNormalizedDataFramesPerClass[k]['emg']:
        varianceDataFrame.loc[ctr] = [df_normalized.var()[0], df_normalized.var()[1], df_normalized.var()[2], df_normalized.var()[3], df_normalized.var()[4], df_normalized.var()[5], df_normalized.var()[6], df_normalized.var()[7]]
        mavDataFrame.loc[ctr] = [robust.mad(df_normalized)[0], robust.mad(df_normalized)[1], robust.mad(df_normalized)[2], robust.mad(df_normalized)[3], robust.mad(df_normalized)[4], robust.mad(df_normalized)[5], robust.mad(df_normalized)[6], robust.mad(df_normalized)[7]]
        meanDataFrame.loc[ctr] = [np.mean(df_normalized)[0], np.mean(df_normalized)[1], np.mean(df_normalized)[2], np.mean(df_normalized)[3], np.mean(df_normalized)[4], np.mean(df_normalized)[5], np.mean(df_normalized)[6], np.mean(df_normalized)[7]]
        sdDataFrame.loc[ctr] = [np.std(df_normalized)[0], np.std(df_normalized)[1], np.std(df_normalized)[2], np.std(df_normalized)[3], np.std(df_normalized)[4], np.std(df_normalized)[5], np.std(df_normalized)[6], np.std(df_normalized)[7]]
        rmsDataFrame.loc[ctr] = [np.sqrt(np.mean(df_normalized[0].values**2)), np.sqrt(np.mean(df_normalized[1].values**2)), np.sqrt(np.mean(df_normalized[2].values**2)), np.sqrt(np.mean(df_normalized[3].values**2)), np.sqrt(np.mean(df_normalized[4].values**2)), np.sqrt(np.mean(df_normalized[5].values**2)), robust.mad(df_normalized)[6], np.sqrt(np.mean(df_normalized[7].values**2))]

        ctr +=1 
        
    print("Done computing the var, mav, mean , sd, and rms of " + str(ctr) + " files")
    listOfVarianceDataFramePerClass.append(varianceDataFrame)
    listOfMavDataFramePerClass.append(mavDataFrame)
    listOfMeanDataFramePerClass.append(meanDataFrame)
    listOfSdDataFramePerClass.append(sdDataFrame)
    listOfRmsDataFramePerClass.append(rmsDataFrame)


Mean and Median Frequency

In [ ]:
import pandas as pd
import numpy as np

listOfMeanFreqDataFramePerClass = []
listOfMedFreqDataFramePerClass = []

for k in listOfNormalizedDataFramesPerClass.keys():
    print("----------------- Word " + k + "-----------------")
    ctr = 0
    
    meanFreqDataFramePerClass = []
    meanFreqDataFrame = pd.DataFrame(columns = ['MeanFreq of EMG1', 'MeanFreq of EMG2', 'MeanFreq of EMG3', 'MeanFreq of EMG4',
                                 'MeanFreq of EMG5', 'MeanFreq of EMG6', 'MeanFreq of EMG7', 'MeanFreq of EMG8'])
    
    medFreqDataFramePerClass = []
    medFreqDataFrame = pd.DataFrame(columns = ['MedianFreq of EMG1', 'MedianFreq of EMG2', 'MedianFreq of EMG3', 'MedianFreq of EMG4',
                                 'MedianFreq of EMG5', 'MedianFreq of EMG6', 'MedianFreq of EMG7', 'MedianFreq of EMG8'])
    
    for df_normalized in listOfNormalizedDataFramesPerClass[k]['emg']:
        meanFreq = []
        medFreq = []

        for i in range(0, 8):
            data = df_normalized[i].values
            ps = np.abs(np.fft.fft(data))**2

            freqs = np.fft.fftfreq(data.size, 1/100)

            sumProd = 0
            sumPs = 0

            for f, p in zip(freqs,ps):
                sumProd += f*p
                sumPs += p

            meanFreq.append(sumProd/sumPs)
            medFreq.append(sumPs/2)
            
        meanFreqDataFrame.loc[ctr] = [meanFreq[0], meanFreq[1], meanFreq[2], meanFreq[3], meanFreq[4], meanFreq[5], meanFreq[6], meanFreq[7]]
        medFreqDataFrame.loc[ctr] = [medFreq[0], medFreq[1], medFreq[2], medFreq[3], medFreq[4], medFreq[5], medFreq[6], medFreq[7]]

        ctr+=1
        
    print("Done getting the Mean and Median Frequency of " + str(ctr) + " files")
    listOfMeanFreqDataFramePerClass.append(meanFreqDataFrame)
    listOfMedFreqDataFramePerClass.append(medFreqDataFrame)

Label Class

In [21]:
dataFramesPerFeature = {}

dataFramesPerFeature['variance'] = listOfVarianceDataFramePerClass[ctr]
dataFramesPerFeature['mav'] = listOfMavDataFramePerClass[ctr]
dataFramesPerFeature['rms'] = listOfRmsDataFramePerClass[ctr]

for k in dataFramesPerFeature.keys():
    print(k)

variance
mav
rms


In [32]:
#print(listOfMavDataFramePerClass[0])
#print(listOfRmsDataFramePerClass[0])
ctr = 0

listOfLabeledDataPerClass = {}
for k in listOfTrimmedDataFramesPerClass.keys():
    dataFramesPerFeature = {}

    dataFramesPerFeature['variance'] = listOfVarianceDataFramePerClass[ctr]
    dataFramesPerFeature['mav'] = listOfMavDataFramePerClass[ctr]
    dataFramesPerFeature['rms'] = listOfRmsDataFramePerClass[ctr]
    dataFramesPerFeature['sd'] = listOfSdDataFramePerClass[ctr]
    dataFramesPerFeature['mean'] = listOfMeanDataFramePerClass[ctr]
    dataFramesPerFeature['meanFreq'] = listOfMedFreqDataFramePerClass[ctr]
    dataFramesPerFeature['meanFreq'] = listOfMeanFreqDataFramePerClass[ctr]
    
    
    for k2 in dataFramesPerFeature.keys():
        if k2 != 'variance':
            for i in dataFramesPerFeature[k2].columns:
                dataFramesPerFeature['variance'][i] = dataFramesPerFeature[k2][i]

    dataFramesPerFeature['variance']['Class'] = k
    listOfLabeledDataPerClass[k] = dataFramesPerFeature['variance']
    ctr+=1
    print("Done labeling emg data of " + k)
 

Done labeling emg data of ano
Done labeling emg data of bibili
Done labeling emg data of black
Done labeling emg data of dito
Done labeling emg data of isa
Done labeling emg data of kape
Done labeling emg data of kayo
Done labeling emg data of kulay
Done labeling emg data of magkano
Done labeling emg data of mayroon
Done labeling emg data of niyo
Done labeling emg data of oras
Done labeling emg data of pagkain
Done labeling emg data of small
Done labeling emg data of tubig


Data Consolidation (save to csv)

In [33]:
newDf = listOfLabeledDataPerClass['ano']
for k in listOfTrimmedDataFramesPerClass.keys():
    if k != 'ano':
        print(k)
        newDf= newDf.append(listOfLabeledDataPerClass[k])
newDf.to_csv('/Users/user/Desktop/consolidatedEMG.csv', encoding='utf-8', index=False)

bibili
black
dito
isa
kape
kayo
kulay
magkano
mayroon
niyo
oras
pagkain
small
tubig


## For IMU values

Trimming (For Acceleration and Gyroscope)

In [ ]:
import pandas as pd

listOfTrimmedAccDataFramesPerClass = {}

for k in listOfDataFramesPerClass.keys():
    print("----------------- Word " + k + "-----------------")
    ctr = 0
    trimmedDataFramesPerClass = []
    dataFrameDictionary = {}

    for d in listOfDataFramesPerClass[k]['acc']:
        keep_col = ['x','y','z']
        new_df = d[keep_col]
        new_df = new_df[:-24]
        trimmed_df = new_df.iloc[24:]
        if len(trimmed_df) < 50:
            print(str(ctr) + " " + str(len(trimmed_df)))
        trimmedDataFramesPerClass.append(trimmed_df)
        ctr += 1
        
    print("Done Trimming " + str(ctr) + " files")
    dataFrameDictionary["acc"] = trimmedDataFramesPerClass
    listOfTrimmedAccDataFramesPerClass[k] = dataFrameDictionary

In [ ]:
import pandas as pd

listOfTrimmedGyroDataFramesPerClass = {}

for k in listOfDataFramesPerClass.keys():
    print("----------------- Word " + k + "-----------------")
    ctr = 0
    trimmedDataFramesPerClass = []
    dataFrameDictionary = {}

    for d in listOfDataFramesPerClass[k]['gyro']:
        keep_col = ['x','y','z']
        new_df = d[keep_col]
        new_df = new_df[:-24]
        trimmed_df = new_df.iloc[24:]
        if len(trimmed_df) < 50:
            print(str(ctr) + " " + str(len(trimmed_df)))
        trimmedDataFramesPerClass.append(trimmed_df)
        ctr += 1
        
    print("Done Trimming " + str(ctr) + " files")
    dataFrameDictionary["gyro"] = trimmedDataFramesPerClass
    listOfTrimmedGyroDataFramesPerClass[k] = dataFrameDictionary

Normalization (For Acceleration and Gyroscope)

In [ ]:
from sklearn import preprocessing

listOfNormalizedAccDataFramesPerClass = {}

for k in listOfTrimmedAccDataFramesPerClass.keys():
    print("----------------- Word " + k + "-----------------")
    ctr = 0
    NormalizedDataFramesPerClass = []
    dataFrameDictionary = {}

    for d in listOfTrimmedAccDataFramesPerClass[k]['acc']:
        #print(str(len(d.index)) + " " + str(ctr))
        # Create x, where x the 'scores' column's values as floats
        x = d.values.astype(float)

        # Create a minimum and maximum processor object
        min_max_scaler = preprocessing.MinMaxScaler()

        # Create an object to transform the data to fit minmax processor
        x_scaled = min_max_scaler.fit_transform(x)

        # Run the normalizer on the dataframe
        df_normalized = pd.DataFrame(x_scaled)
        #df_normalized
        NormalizedDataFramesPerClass.append(df_normalized)
        ctr += 1
        
    print("Done Normalizing " + str(ctr) + " files")
    dataFrameDictionary["acc"] = NormalizedDataFramesPerClass
    listOfNormalizedAccDataFramesPerClass[k] = dataFrameDictionary  
    
print("Data Normalization done!!")



In [ ]:
from sklearn import preprocessing

listOfNormalizedGyroDataFramesPerClass = {}

for k in listOfTrimmedGyroDataFramesPerClass.keys():
    print("----------------- Word " + k + "-----------------")
    ctr = 0
    NormalizedDataFramesPerClass = []
    dataFrameDictionary = {}

    for d in listOfTrimmedGyroDataFramesPerClass[k]['gyro']:
        #print(str(len(d.index)) + " " + str(ctr))
        # Create x, where x the 'scores' column's values as floats
        x = d.values.astype(float)

        # Create a minimum and maximum processor object
        min_max_scaler = preprocessing.MinMaxScaler()

        # Create an object to transform the data to fit minmax processor
        x_scaled = min_max_scaler.fit_transform(x)

        # Run the normalizer on the dataframe
        df_normalized = pd.DataFrame(x_scaled)
        #df_normalized
        NormalizedDataFramesPerClass.append(df_normalized)
        ctr += 1
        
    print("Done Normalizing " + str(ctr) + " files")
    dataFrameDictionary["gyro"] = NormalizedDataFramesPerClass
    listOfNormalizedGyroDataFramesPerClass[k] = dataFrameDictionary  
    
print("Data Normalization done!!")



Label Class

In [ ]:
listOfNormalizedDataFramesPerClass['ano']['imu'][0]

In [ ]:
#print(listOfMavDataFramePerClass[0])
#print(listOfRmsDataFramePerClass[0])

listOfLabeledImuDataPerClass = []
for k in listOfTrimmedAccDataFramesPerClass.keys():
    ctr = 0

    labeledDataFramePerClass = []
    labeledDataFrame = pd.DataFrame(columns = ['X of Accelerometer', 'Y of Accelerometer', 'Z of Accelerometer', 
                                               'X of Gyroscope', 'Y of Gyroscope', 'Z of Gyroscope', 'Class'])
    for a, g in zip(listOfNormalizedAccDataFramesPerClass[k]['acc'],listOfNormalizedGyroDataFramesPerClass[k]['gyro'])
        labeledDataFrame.loc[ctr] = [a[0], a[1], a[2], g[0], g[1], g[2], k]
        ctr+=1

    listOfLabeledImuDataPerClass.append(labeledDataFrame)
    print("Done labeling imu data of " + k)


In [ ]:
import pandas as pd

listOfMavDataFramePerClass = []

for k in listOfNormalizedDataFramesPerClass.keys():
    print("----------------- Word " + k + "-----------------")
    ctr = 0
    
    mavDataFramePerClass = []
    mavDataFrame = pd.DataFrame(columns = ['Mav of EMG1', 'Mav of EMG2', 'Mav of EMG3', 'Mav of EMG4',
                                 'Mav of EMG5', 'Mav of EMG6', 'Mav of EMG7', 'Mav of EMG8'])
   
    for df_normalized in listOfNormalizedDataFramesPerClass[k]['emg']:
        mavDataFrame.loc[ctr] = [robust.mad(df_normalized)[0], robust.mad(df_normalized)[1], robust.mad(df_normalized)[2], robust.mad(df_normalized)[3], robust.mad(df_normalized)[4], robust.mad(df_normalized)[5], robust.mad(df_normalized)[6], robust.mad(df_normalized)[7]]
        ctr +=1 
        
    print("Done getting the mav of " + str(ctr) + " files")
    listOfMavDataFramePerClass.append(mavDataFrame)

Data Consolidation (save to csv)

In [ ]:
newDf = listOfLabeledDataPerClass['ano']
for k in listOfTrimmedDataFramesPerClass.keys():
    if k != 'ano':
        print(k)
        newDf= newDf.append(listOfLabeledDataPerClass[k])
newDf.to_csv('/Users/user/Desktop/consolidatedEMG.csv', encoding='utf-8', index=False)

Kurtosis (For Acceleration)

In [ ]:
# per row returns nan, nan, nan...
df_normalized.kurtosis().tolist()

Standard Deviation (For Acceleration)

In [ ]:
df_normalized.std(axis=1).tolist()

Mean (For Acceleration)

In [ ]:
df_normalized.mean(axis=1).tolist()

Variance (For Acceleration)

In [ ]:
df_normalized.var(axis=1).tolist()

Entropy (For Acceleration)

In [ ]:
import scipy as sp
# per row returns 0...
sp.stats.entropy(df_normalized).tolist()

Skewness (For Acceleration)

In [ ]:
df_normalized.skew(axis=1).tolist()

Mean Absolute Value (For Acceleration)

In [ ]:
from statsmodels import robust

rows = []
mavValues = []
length = df_normalized.shape[0]

for x in range(0, length):
    rows.append(df_normalized.iloc[x].tolist())

for x in range(0, length):
    mavValues.append(robust.mad(rows[x]))


Root Mean Square (For Acceleration)

In [ ]:
import numpy as np

np.sqrt(df_normalized.mean(axis=1)**2).tolist()

# Classification-SVM

In [ ]:
# Support Vector Machine
from sklearn import datasets
from sklearn import metrics
from sklearn.svm import SVC
# load the iris datasets
dataset = datasets.load_iris()
# fit a SVM model to the data
model = SVC()
model.fit(dataset.data, dataset.target)
print(model)
# make predictions
expected = dataset.target
predicted = model.predict(dataset.data)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))